เพื่อที่จะรันโน้ตบุ๊กต่อไปนี้ หากคุณยังไม่ได้ทำ คุณจำเป็นต้องตั้งค่า openai key ในไฟล์ .env เป็น `OPENAI_API_KEY`


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

API_KEY = os.getenv("OPENAI_API_KEY","")
assert API_KEY, "ERROR: OpenAI Key is missing"

client = OpenAI(
    api_key=API_KEY
    )

model = 'text-embedding-ada-002'

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

ต่อไป เราจะโหลด Embedding Index ลงใน Pandas Dataframe โดย Embedding Index จะถูกเก็บไว้ในไฟล์ JSON ชื่อว่า `embedding_index_3m.json` Embedding Index นี้ประกอบด้วย Embeddings สำหรับแต่ละไฟล์ถอดเสียงของ YouTube จนถึงปลายเดือนตุลาคม 2023


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

ต่อไป เราจะสร้างฟังก์ชันชื่อ `get_videos` ที่จะใช้ค้นหา Embedding Index ตามคำค้นหา ฟังก์ชันนี้จะคืนค่าวิดีโอ 5 อันดับแรกที่มีความคล้ายคลึงกับคำค้นหามากที่สุด วิธีการทำงานของฟังก์ชันนี้มีดังนี้:

1. ขั้นแรก จะมีการสร้างสำเนาของ Embedding Index ขึ้นมา
2. จากนั้นจะคำนวณ Embedding สำหรับคำค้นหาโดยใช้ OpenAI Embedding API
3. ต่อไปจะมีการสร้างคอลัมน์ใหม่ใน Embedding Index ชื่อว่า `similarity` โดยคอลัมน์ `similarity` นี้จะเก็บค่าความคล้ายคลึงแบบ cosine ระหว่าง Embedding ของคำค้นหาและ Embedding ของแต่ละช่วงวิดีโอ
4. หลังจากนั้น Embedding Index จะถูกกรองโดยใช้คอลัมน์ `similarity` โดยจะเลือกเฉพาะวิดีโอที่มีค่า cosine similarity มากกว่าหรือเท่ากับ 0.75 เท่านั้น
5. สุดท้าย Embedding Index จะถูกจัดเรียงตามคอลัมน์ `similarity` และคืนค่าวิดีโอ 5 อันดับแรก


In [ ]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

ฟังก์ชันนี้ง่ายมาก เพียงแค่แสดงผลลัพธ์ของการค้นหา


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. ขั้นแรก Embedding Index จะถูกโหลดเข้าไปใน Pandas Dataframe
2. จากนั้นผู้ใช้จะถูกขอให้กรอกคำค้นหา
3. ต่อไปจะมีการเรียกใช้ฟังก์ชัน `get_videos` เพื่อค้นหา Embedding Index ด้วยคำค้นหานั้น
4. สุดท้ายจะมีการเรียกใช้ฟังก์ชัน `display_results` เพื่อแสดงผลลัพธ์ให้กับผู้ใช้
5. จากนั้นผู้ใช้จะถูกขอให้กรอกคำค้นหาใหม่อีกครั้ง กระบวนการนี้จะดำเนินต่อไปจนกว่าผู้ใช้จะพิมพ์ `exit`

![](../../../../translated_images/notebook-search.1e320b9c7fcbb0bc1436d98ea6ee73b4b54ca47990a1c952b340a2cadf8ac1ca.th.png)

คุณจะถูกขอให้กรอกคำค้นหา ให้พิมพ์คำค้นหาของคุณแล้วกด Enter แอปพลิเคชันจะแสดงรายการวิดีโอที่เกี่ยวข้องกับคำค้นหานั้นให้คุณ พร้อมทั้งลิงก์ไปยังช่วงเวลาที่มีคำตอบในวิดีโอ

ตัวอย่างคำค้นหาที่คุณสามารถลองได้:

- Azure Machine Learning คืออะไร
- convolutional neural networks ทำงานอย่างไร
- neural network คืออะไร
- ฉันสามารถใช้ Jupyter Notebooks กับ Azure Machine Learning ได้ไหม
- ONNX คืออะไร


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from imput
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)


---

**ข้อจำกัดความรับผิดชอบ**:  
เอกสารฉบับนี้ได้รับการแปลโดยใช้บริการแปลภาษา AI [Co-op Translator](https://github.com/Azure/co-op-translator) แม้ว่าเราจะพยายามอย่างเต็มที่เพื่อความถูกต้อง แต่โปรดทราบว่าการแปลโดยระบบอัตโนมัติอาจมีข้อผิดพลาดหรือความไม่ถูกต้อง เอกสารต้นฉบับในภาษาต้นทางควรถือเป็นแหล่งข้อมูลที่เชื่อถือได้ สำหรับข้อมูลที่มีความสำคัญ แนะนำให้ใช้บริการแปลโดยนักแปลมืออาชีพ ทางเราจะไม่รับผิดชอบต่อความเข้าใจผิดหรือการตีความที่คลาดเคลื่อนซึ่งเกิดจากการใช้การแปลนี้
